# Генерация игровых диалогов с помощью больших языковых моделей

Диалог — один из ключевых инструментов повествования в видеоиграх. Через него игрок узнаёт мир, персонажей и последствия собственных решений.

В этом ноутбуке мы исследуем, как большие языковые модели могут использоваться для генерации игровых диалогов между главным героем и NPC. Мы рассмотрим, каким образом можно задавать контекст сцены, описывать персонажей и получать на выходе связные, вариативные и управляемые диалоги, пригодные для использования в игровом прототипе.

## Постановка задачи
В данном ноутбуке будет решаться задача генерации дерева диалога главного героя с NPC.

Важно, что наша команда отвечает только за логику генерации диалогов, поэтому входные параметры и формат выхода зафиксирован договорённостями с другими командами. 

Ограничения:
- Задачу нужно решить с помощью одного запроса в LLM
- В качестве входных параметров персонажей и игрового окружения должен быть использован набор параметров, указанный в [файле](models.py).
- В результате работы сервиса генерации должен получиться JSON со следующей структурой:
```python
{
    "data": [
        {
            "id": id_вершины,
            "info": "Краткое содержание текста в вершине",
            "line": "Текст в вершине",
            "to": [
                {
                    "id": id_вершины_в_котороую_ведёт_ребро,
                    "info": "Краткое содержание текста в ребре",
                    "line": "Текст в ребре"
                }
            ] # список вершин, куда идут рёбра
        }
    ]
}
```

## Решение

### Генерация дерева диалога

In [6]:
from openai import OpenAI

In [ ]:
# Создаём клиент для запросов к API deepseek
client = OpenAI(
    api_key="sk-b4d8c96a0b984582ad8ecf6fe...",
    base_url="https://api.deepseek.com"
)

Чтобы в результате работы модели получить структурированный ответ (в нашем случае JSON), будем использовать 2 подхода:
1. Явно укажем в промпте структуру ответа.
2. В параметрах вызова модели укажем, что нужен структурированный ответ.

Кроме того, будем использовать готовые классы для хранения параметров мира и героев

In [8]:
from models import GameParameters, DialogParamters, CharacterParameters

In [9]:
from string import Template


system_prompt = ""
# В промпте для модели явно должна содержаться структура ответа, которую нам нужно получить
# Кроме того, будем использовать шаблоны, чтобы можно было удобнее подставлять параметры
# главного героя и NPC. В шаблоне для обозначения переменной используется синтаксис $var_name
prompt_str = '''
Сгенерируй пример диалога между NPC и главным героем. Диалог представь в 
виде графа, где вершины — реплики NPC, а рёбра — варианты реплик главного героя.

Главного героя зовут: $mc_name
NPC зовут: $npc_name
В ответе приведи только json со следующей структурой:
```
{
    "data": [
        {
            "id": id_вершины,
            "info": "Краткое содержание текста в вершине",
            "line": "Текст в вершине",
            "to": [
                {
                    "id": id_вершины_в_которое_ведёт_ребро,
                    "info": "Краткое содержание текста в ребре",
                    "line": "Текст в ребре"
                }
            ] # список вершин, куда идут рёбра
        }
    ]
}
```
'''

prompt_temp = Template(prompt_str) # формируем шаблон из строки

mc_parameters = CharacterParameters(name="Вася")
npc_parameters = CharacterParameters(name="НеВася")

prompt = prompt_temp.safe_substitute(
    mc_name=mc_parameters.name,
    npc_name=npc_parameters.name
)



In [10]:
# Создаём запрос к API
response = client.chat.completions.create(
    model = "deepseek-chat", # выбираем модель
    messages=[
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": prompt},
    ], # промпты
    stream=False,
    max_tokens=8192, # максимальное число токенов в ответе
    response_format={
        'type': "json_object"
    } # требуем ответ в json-формате
)

In [11]:
# Получившийся ответ
print(response.choices[0].message.content)

{
    "data": [
        {
            "id": 1,
            "info": "НеВася приветствует Васю",
            "line": "Привет, Вася! Как дела?",
            "to": [
                {
                    "id": 2,
                    "info": "Вася отвечает вежливо",
                    "line": "Привет, НеВася! Всё хорошо, спасибо."
                },
                {
                    "id": 3,
                    "info": "Вася отвечает грубо",
                    "line": "Отстань, НеВася."
                }
            ]
        },
        {
            "id": 2,
            "info": "НеВася предлагает помощь",
            "line": "Рад слышать! Могу ли я чем-то помочь?",
            "to": [
                {
                    "id": 4,
                    "info": "Вася принимает помощь",
                    "line": "Да, пожалуйста, подскажи, как пройти к замку."
                },
                {
                    "id": 5,
                    "info": "Вася отказывается от помощи",
   

### Визуализация графа диалога

Для оценки качества получившихся диалогов удобно их визуализировать в виде интерактивного графа.

In [ ]:
from pyvis.network import Network
from textwrap import fill
import json

def format_tooltip(text: str, width: int = 80) -> str:
    if not text:
        return ""
    lines = []
    for line in text.splitlines():
        lines.extend(fill(line, width).splitlines())
    return "\n".join(lines)


def build_pyvis_graph(graph_data: dict, output_html: str = "graph.html"):
    net = Network(
        height="800px",
        width="100%",
        directed=True,
    )

    net.set_options("""
    {
      "interaction": {
        "hover": true,
        "tooltipDelay": 200
      },
      "edges": {
        "arrows": { "to": { "enabled": true } }
      },
      "physics": {
        "barnesHut": {
          "gravitationalConstant": -30000,
          "springLength": 200
        }
      }
    }
    """)

    # --- вычисляем финальные ноды (без исходящих рёбер) ---
    all_ids = {node["id"] for node in graph_data["data"]}
    has_outgoing = set()

    for node in graph_data["data"]:
        for edge in node.get("to", []):
            has_outgoing.add(node["id"])

    final_nodes = all_ids - has_outgoing

    TOP_X = 0
    TOP_Y = -400

    # --- добавление вершин ---
    for i, node in enumerate(graph_data["data"]):
        node_id = node["id"]

        is_root = i == 0
        is_final = node_id in final_nodes

        color = None
        if is_root:
            color = {
                "background": "#4CAF50",  # зелёный
                "border": "#2E7D32",
            }
        elif is_final:
            color = {
                "background": "#E53935",  # красный
                "border": "#B71C1C",
            }

        net.add_node(
            node_id,
            label=node.get("info", str(node_id)),
            title=format_tooltip(node.get("line")),
            shape="box",
            color=color,
            x=TOP_X if is_root else None,
            y=TOP_Y if is_root else None,
            fixed={"x": True, "y": True} if is_root else False,
        )

    # --- рёбра ---
    for node in graph_data["data"]:
        for edge in node.get("to", []):
            net.add_edge(
                node["id"],
                edge["id"],
                label=edge.get("info", ""),
                title=format_tooltip(edge.get("line")),
            )

    net.write_html(output_html, open_browser=True)


In [ ]:
graph_data = json.loads(response.choices[0].message.content)
build_pyvis_graph(graph_data)

## Пример промпта, решающего нашу задачу

In [ ]:
prompt_correct = '''
Создай диалог для компьютерной игры в виде списка смежности ориентированного ациклического графа. Учитывай следующие требования:
<Характеристики структуры>
	<Структура тип=вершина>
		<Типы вершин>
			- Тип C (Choice Nodes) - вершины, в которых у игрока есть от 2 до 4 вариантов ответа. Ответы в C-вершинах **должны** влиять на сюжет/отношения 
			- Тип M (Monologue Nodes) - вершины, где NPC раскрывает характер/историю через монолог. Игрок **только слушает** (**единственный** вариант ответа: "Продолжить"). Каждый M-узел имеет выходную степень 1. Лимит: **не более 2 M-узлов подряд** в любой ветке. Обязательно **проверь** что в итоговом графе **нет более 2 M-узлов подряд**.
			- Тип P (Pendant Nodes) - вершины, в которых заканчивается диалог. Каждая P-вершина имеет выходную степень 0. Число реплик NPC до каждого P-узла должно лежать в диапазоне от 5 до 10.
		</Типы вершин>
		<Формат>
            В ответе приведи только JSON следующего формата:
            ```
            {
                "data": [
                    {
                        "id": id_вершины,
                        "info": "Краткое содержание текста в вершине",
                        "line": "Текст в вершине (тут только текст, без имени и эмоций говорящего)",
                        "type": 'Тип вершины'
                        "to": [
                            {
                                "id": id_вершины_в_которое_ведёт_ребро,
                                "info": "Краткое содержание текста в ребре",
                                "line": "Текст в ребре"
                            }
                        ] # список вершин, куда идут рёбра
                    }
                ]
            }
            ```
		</Формат>
</Характеристики структуры> 
<Характеристики диалога>
	<Характеристика тип=NPC>
		<Имя>Макс Планк</Имя>
		<Стиль речи>Строгий, логичный, консервативный</Стиль речи>
		<Профессия>Теоретический физик, профессор университета</Профессия>
		<Внешний вид>$NPC_look</Внешний вид>
		<Взаимоотношения с игроком>Отношение NPC к игроку - незнаком</Взаимоотношения с игроком>
		<Черты характера>Черезвычайно умный, добрый</Черты характера>
		<Дополнительная информация>Спешит писать статью</Дополнительная информация>
	</Характеристика тип=NPC>
	<Характеристика тип=игрок>
		<Имя>Вася</Имя>
		<Стиль речи>Быстрый, немного спутанный</Стиль речи>
		<Профессия>Школьник</Профессия>
		<Внешний вид>Слегка небрежный</Внешний вид>
		<Взаимоотношения с NPC>Отношение игрока к NPC - знает, но лично не знаком</Взаимоотношения с NPC>
		<Черты характера>Впечатлительный, умный</Черты характера>
		<Дополнительная информация>Очень рад встретить великого физика</Дополнительная информация>
	</Характеристика тип=игрок>	
	<Характеристика тип=окружение> 
		**Обязательно** учитывай характеристики окружения, в котором происходят события диалога: Холл университета
	</Характеристика тип=окружение>
	<Характеристика тип=игровой мир>
		<Жанр>Фантастика</Жанр>
		<Исторический период>Индустриальный</Исторический период>
		<Тональность>Комедия</Тональность>
		<Описание>Герой попал в прошлое и пытается вернуться домой, для этого ему нужно помочь 5 великим учёным сделать их открытия</Описание>
	</Характеристика тип=игровой мир>
	<Контекст диалога>
    В этом поле содержится описание краткой предыстории и ключевых событий, происходящих в диалоге. **Учитывай** их при генерации структуры:
    Вася отлично знает физику, но проспал урок про Макса Планка, поэтому не знает, какое открытие он сделал. Но если в диалоге сможет понять,
    то сможет помочь.
    </Контекст диалога>
</Характеристики диалога>
<Инструкции>
	<Инструкции тип=диалог>
		При генерации тематик диалога **строго** соблюдай следующие инструкции:
		- Ты **обязан** быть естественным и учитывать, что NPC и игрок могут менять тему, но без резких скачков.
		- Некоторые линии диалогов **могут** не привести к достижению поставленной цели.
	</Инструкции тип=диалог>
	<Инструкции тип=структура>
		При генерации структуры графа **строго** соблюдай следующие инструкции:
		- Ты **обязан** соблюдать все требования по структурам данных и учитывать все характеристики
		- P-вершины **должны** завершать диалог логично
		- **Обязательно проверь**, что **нет** рёбер, ведущих в несуществующие вершины.
		- Полученный граф **должен** быть связным
	</Инструкции тип=структура>
	В ответ верни **только JSON** без пояснений, сохраняя все поля из примера
</Инструкции>	
Строго соблюдай все требования. Сделай диалог логичным и интересным. **Перепроверь**, выполнил ли ты все требования и инструкции.
Теперь создай диалог:
'''